In [2]:
import numpy as np
import torch

# PyTorch テンソルの表示オプションを設定することで、
# print 時の視認性と簡潔さを向上させる

torch.set_printoptions(
    edgeitems=2,  # 各次元の先頭と末尾の要素数（中間は "..." で省略）
    precision=2,  # 浮動小数点数の小数点以下の桁数（例: 0.123456 → 0.12）
    linewidth=75,  # 1行あたりの最大表示幅（長すぎる行は改行される）
)

In [3]:
import csv
import numpy as np

# ワイン品質データセット（白ワイン）のCSVファイルのパス
wine_path = "../../data/p1ch4/tabular-wine/winequality-white.csv"

# CSVファイルをNumPy配列として読み込む
# - dtype=np.float32: すべての値をfloat32型として読み込む
# - delimiter=";": 区切り文字はセミコロン（このCSVはExcel形式）
# - skiprows=1: ヘッダー行（列名）をスキップ
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";", skiprows=1)

# 読み込んだデータを表示（shapeや内容を確認）
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]],
      shape=(4898, 12), dtype=float32)

In [4]:
# csv.reader でファイルを開き、最初の行（=列名の行）だけを取得
col_list = next(csv.reader(open(wine_path), delimiter=";"))

# データ本体はすでに NumPy 配列として読み込まれていると仮定
# wineq_numpy = np.loadtxt(...)

# データの形状と列名一覧を確認
wineq_numpy.shape, col_list

((4898, 12),
 ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol',
  'quality'])

In [5]:
wineq = torch.from_numpy(wineq_numpy)

wineq.shape, wineq.dtype

(torch.Size([4898, 12]), torch.float32)

In [6]:
data = wineq[:, :-1]
data, data.shape

(tensor([[ 7.00,  0.27,  ...,  0.45,  8.80],
         [ 6.30,  0.30,  ...,  0.49,  9.50],
         ...,
         [ 5.50,  0.29,  ...,  0.38, 12.80],
         [ 6.00,  0.21,  ...,  0.32, 11.80]]),
 torch.Size([4898, 11]))

In [7]:
target = wineq[:, -1]
target, target.shape

(tensor([6., 6.,  ..., 7., 6.]), torch.Size([4898]))

In [8]:
target = wineq[:, -1].long()
target

tensor([6, 6,  ..., 7, 6])

In [9]:
# 例: target = tensor([3, 1, 4, 0, ...]) のような整数クラスラベル（0〜9）

# ワンホット形式を格納するテンソルを初期化（全ゼロ）
# shape: [サンプル数, クラス数] → 10クラス分類タスクを想定
target_onehot = torch.zeros(target.shape[0], 10)

# target.unsqueeze(1) によって shape を [N, 1] に拡張し、
# scatter_ により 1.0 を指定インデックスに代入（in-place操作）
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

tensor([[0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.],
        ...,
        [0., 0.,  ..., 0., 0.],
        [0., 0.,  ..., 0., 0.]])

In [10]:
# 例: target = tensor([2, 0, 4]) のような shape: [N]（1次元ベクトル）を仮定

# 次元1（列方向）に1次元を追加し、shape を [N, 1] にする
# これは scatter_ や loss 計算（CrossEntropyLossなど）で必要になる形式
target_unsqueezed = target.unsqueeze(1)

# 確認
target_unsqueezed

tensor([[6],
        [6],
        ...,
        [7],
        [6]])